In [1]:
import os
import urllib.request
import zipfile
import torch
import torch.nn as nn
import torch.optim as optim
from pointnet import PointNetClassifier
from ModelNetDataLoader import ModelNetDataLoader

# Download and extract the ModelNet40 dataset
url = "https://shapenet.cs.stanford.edu/media/modelnet40_ply_hdf5_2048.zip"
filename = "modelnet40_ply_hdf5_2048.zip"
if not os.path.exists(filename):
    print("Downloading ModelNet40 dataset...")
    urllib.request.urlretrieve(url, filename)
    with zipfile.ZipFile(filename, "r") as zip_ref:
        zip_ref.extractall(".")

# Load and preprocess 3D point cloud data
train_data = ModelNetDataLoader("modelnet40_ply_hdf5_2048/ply_data_train.h5", num_points=1024)
test_data = ModelNetDataLoader("modelnet40_ply_hdf5_2048/ply_data_test.h5", num_points=1024)

# Create a PointNet classifier
model = PointNetClassifier(num_classes=40)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for batch_data, batch_labels in train_data:
        # Forward pass
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Print training progress
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Save the trained model
torch.save(model.state_dict(), "pointnet_model.pth")

# Test driver
model.eval()
class_names = ["airplane", "bathtub", "bed", "bench", "bookshelf", "bottle", "bowl", "car", "chair", "cone",
               "cup", "curtain", "desk", "door", "dresser", "flower_pot", "glass_box", "guitar", "keyboard",
               "lamp", "laptop", "mantel", "monitor", "night_stand", "person", "piano", "plant", "radio",
               "range_hood", "sink", "sofa", "stairs", "stool", "table", "tent", "toilet", "tv_stand", "vase",
               "wardrobe", "xbox"]

# Sample test images from online sources
test_images = [
    "https://github.com/charlesq34/pointnet/raw/master/misc/airplane.ply",
    "https://github.com/charlesq34/pointnet/raw/master/misc/chair.ply",
    "https://github.com/charlesq34/pointnet/raw/master/misc/car.ply"
]

for image_url in test_images:
    # Download the test image
    filename = os.path.basename(image_url)
    urllib.request.urlretrieve(image_url, filename)
    
    # Load and preprocess the test image
    test_data = ModelNetDataLoader(filename, num_points=1024)
    test_data = next(iter(test_data))
    
    # Perform inference
    with torch.no_grad():
        outputs = model(test_data)
        _, predicted = torch.max(outputs.data, 1)
    
    predicted_class = class_names[predicted.item()]
    print(f"Test Image: {filename}")
    print(f"Predicted Class: {predicted_class}")
    print("---")
    
    # Clean up the downloaded test image
    os.remove(filename)

ModuleNotFoundError: No module named 'torch'

In [2]:
1pip install torch

SyntaxError: invalid syntax (1599436247.py, line 1)